In [1]:
pip install faker

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import requests
from faker import Faker
import random
import numpy as np

In [3]:
# download taxi dataset for 01-2024
url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet"
response = requests.get(url)
file_path = 'tripdata_2024_01.parquet'

if response.status_code == 200:
    with open(file_path, 'wb') as file:
        file.write(response.content)
    print(f'File downloaded successfully for tripdata_2024_01')
else:
    print('Failed to download file')

File downloaded successfully for tripdata_2024_01


In [4]:
# download zone dataset
zone_url = "https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv"
response = requests.get(zone_url)
file_path = 'zone_dataset.csv'

if response.status_code == 200:
    with open(file_path, 'wb') as file:
        file.write(response.content)
    print(f'File downloaded successfully for zone_dataset')
else:
    print('Failed to download file')

File downloaded successfully for zone_dataset


In [5]:
# download vehicle dataset
vehicle_url = "https://data.cityofchicago.org/api/views/tfm3-3j95/rows.csv"
response = requests.get(vehicle_url)
file_path = 'vehicle_dataset.csv'

if response.status_code == 200:
    with open(file_path, 'wb') as file:
        file.write(response.content)
    print(f'File downloaded successfully for vehicle_dataset')
else:
    print('Failed to download file')

File downloaded successfully for vehicle_dataset


In [6]:
# Read parquet file into DataFrames
# I use only 1 month which already contains about 2 millions records for taxi data
df_taxi = pd.read_parquet('tripdata_2024_01.parquet')
df_user = pd.read_csv('user_data.csv')
df_vehicle = pd.read_csv('vehicle_dataset.csv')
df_zone = pd.read_csv('zone_dataset.csv')

In [7]:
df_taxi.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


In [8]:
df_taxi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2964624 entries, 0 to 2964623
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [9]:
# make sure that pickup and dropoff are in datetime format
df_taxi['tpep_pickup_datetime'] = pd.to_datetime(df_taxi['tpep_pickup_datetime'])
df_taxi['tpep_dropoff_datetime'] = pd.to_datetime(df_taxi['tpep_dropoff_datetime'])

In [10]:
df_taxi = df_taxi.drop_duplicates().reset_index(drop=True)
df_taxi['trip_id'] = df_taxi.index

In [11]:
df_taxi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2964624 entries, 0 to 2964623
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [12]:
df_taxi = df_taxi.dropna()

In [13]:
df_taxi.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,trip_id
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0,0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0,1
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0,2
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0,3
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0,4


In [14]:
df_user.head() 

,User ID,Subscription Type,Monthly Revenue,Join Date,Last Payment Date,Country,Age,Gender,Device,Plan Duration
0,1,Basic,10,15-01-22,10-06-23,United States,28,Male,Smartphone,1 Month
1,2,Premium,15,05-09-21,22-06-23,Canada,35,Female,Tablet,1 Month
2,3,Standard,12,28-02-23,27-06-23,United Kingdom,42,Male,Smart TV,1 Month
3,4,Standard,12,10-07-22,26-06-23,Australia,51,Female,Laptop,1 Month
4,5,Basic,10,01-05-23,28-06-23,Germany,33,Male,Smartphone,1 Month


In [15]:
df_user = df_user.drop_duplicates().reset_index(drop=True)

In [16]:
df_user.isnull().sum()

User ID              0
Subscription Type    0
Monthly Revenue      0
Join Date            0
Last Payment Date    0
Country              0
Age                  0
Gender               0
Device               0
Plan Duration        0
dtype: int64

In [17]:
# we need user_id, age, gender for user and driver 
df_user_new = df_user[['User ID', 'Age', 'Gender']].copy()

In [18]:
df_user_new.head()

,User ID,Age,Gender
0,1,28,Male
1,2,35,Female
2,3,42,Male
3,4,51,Female
4,5,33,Male


In [19]:
# Add data to user dataset 
# Initialize Faker
fake = Faker('en_GB') # for UK

def generate_info(func, n):
    generated = set()
    unique_value = []
    while len(unique_value) < n:
        value = func()
        if value not in generated:
            unique_value.append(value)
            generated.add(value)
    return unique_value

# Generate fake data for each user
df_user_new['name'] = generate_info(fake.name, len(df_user_new))
df_user_new['phone_number'] = generate_info(fake.phone_number, len(df_user_new))
df_user_new['email'] = generate_info(fake.email, len(df_user_new))

In [20]:
# Create data for users
user_df = df_user_new[(df_user_new['User ID'] >= 1 ) & (df_user_new['User ID'] <= 700)].copy()

# Create data for drivers
driver_df = df_user_new[(df_user_new['User ID'] >= 701) & (df_user_new['User ID'] <= 1000)].copy()

In [21]:
# Set the User ID for Driver
driver_df['User ID'] = range(1, len(driver_df)+1)
driver_df = driver_df.reset_index(drop=True)

In [22]:
# We got final user, driver dataset
user_df = user_df.rename({'User ID':'client_id'}, axis=1)
driver_df = driver_df.rename({'User ID':'driver_id','Age':'age', 'Gender':'gender'}, axis=1)

In [23]:
user_df.head()

,client_id,Age,Gender,name,phone_number,email
0,1,28,Male,Helen Kaur-Wilson,+441314960844,irene43@example.com
1,2,35,Female,Lewis Graham,+44118 4960871,guy46@example.com
2,3,42,Male,Gail Gilbert,(0117) 4960459,rachel34@example.org
3,4,51,Female,Jason Dawson,+44114 4960480,hannahstephenson@example.com
4,5,33,Male,Dr Oliver Cooper,+44(0)3069990086,shahrhys@example.com


In [24]:
# Cleaning vehicle data
df_vehicle = df_vehicle.drop_duplicates().dropna()

In [25]:
driver_df.head()

,driver_id,age,gender,name,phone_number,email
0,1,45,Male,Mr Owen Parker,+44909 8790253,susan71@example.com
1,2,36,Male,Owen Hughes,(0115) 496 0684,mohammadmarshall@example.org
2,3,35,Male,Joseph Williams,(0306) 999 0755,vbegum@example.com
3,4,27,Female,Kim Gibson-Thorpe,+441184960579,marshallruth@example.com
4,5,32,Male,Dr Martyn Harris,(0161) 496 0297,marilyn23@example.com


In [26]:
# Selecting column for our vehicle dataset
vehicle_df = df_vehicle[['Public Vehicle Number', 'Vehicle Make', 'Vehicle Model', 'Vehicle Model Year', 'Vehicle Color']].copy()

In [27]:
# Rename column
vehicle_df = vehicle_df.rename({'Public Vehicle Number': 'vehicle_number', 'Vehicle Make':'vehicle_make', 'Vehicle Model': 'vehicle_model', 'Vehicle Model Year': 'vehicle_model_year', 'Vehicle Color': 'vehicle_color'}, axis=1)

In [28]:
# Selecting number vehicle to match with the number of driver which is 300 records
vehicle_df = vehicle_df.sample(n = 300, random_state=1).reset_index(drop=True)

In [29]:
# Assign driver id to each vehicle
vehicle_df['driver_id'] = range(1, len(vehicle_df)+1)

In [30]:
vehicle_df['vehicle_model_year'] = vehicle_df['vehicle_model_year'].astype(int)

In [31]:
vehicle_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   vehicle_number      300 non-null    int64 
 1   vehicle_make        300 non-null    object
 2   vehicle_model       300 non-null    object
 3   vehicle_model_year  300 non-null    int64 
 4   vehicle_color       300 non-null    object
 5   driver_id           300 non-null    int64 
dtypes: int64(3), object(3)
memory usage: 14.2+ KB


In [32]:
vehicle_df.head()

,vehicle_number,vehicle_make,vehicle_model,vehicle_model_year,vehicle_color,driver_id
0,1530,TOYOTA,CAMRY,2020,WHITE,1
1,1787,TOYOTA,PRIUS,2017,GRAY/WHITE,2
2,4013,NISSAN,ALTIMA,2011,WHITE,3
3,6470,TOYOTA,CAMRY,2019,WHITE,4
4,5669,FORD,ESCAPE,2012,WHITE,5


In [33]:
# Create Payment dataset
payment_df = df_taxi[['fare_amount','mta_tax', 'extra', 'improvement_surcharge', 'tip_amount', 'tolls_amount', 'Airport_fee', 'congestion_surcharge']].copy()

In [34]:
# Add payment_id for each record
payment_df['payment_id'] = range(1, len(payment_df)+1)

In [35]:
new_col = ['payment_id'] + [col for col in payment_df.columns if col != 'payment_id']
payment_df = payment_df[new_col]
payment_df.head()

,payment_id,fare_amount,mta_tax,extra,improvement_surcharge,tip_amount,tolls_amount,Airport_fee,congestion_surcharge
0,1,17.7,0.5,1.0,1.0,0.00,0.0,0.0,2.5
1,2,10.0,0.5,3.5,1.0,3.75,0.0,0.0,2.5
2,3,23.3,0.5,3.5,1.0,3.00,0.0,0.0,2.5
3,4,10.0,0.5,3.5,1.0,2.00,0.0,0.0,2.5
4,5,7.9,0.5,3.5,1.0,3.20,0.0,0.0,2.5


In [36]:
# Create Trip dataset 
trip_df = df_taxi[['trip_id', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'PULocationID', 'DOLocationID']].copy()

In [37]:
trip_df = trip_df.rename({'tpep_pickup_datetime':'pickup_datetime', 'tpep_dropoff_datetime': 'dropoff_datetime', 'DOLocationID':'DO_loc_id', 'PULocationID':'PU_loc_id'}, axis=1)

In [38]:
trip_df['payment_id'] = payment_df['payment_id']

In [39]:
trip_df['trip_id'] = range(1, len(trip_df)+1)

In [40]:
trip_df['driver_id'] = np.random.choice(driver_df['driver_id'], size=len(trip_df))
trip_df['client_id'] = np.random.choice(user_df['client_id'], size=len(trip_df))
trip_df['rating'] = np.random.choice(['positive','negative','neutral'], size=len(trip_df))

In [41]:
# check number of client and driver 
a = trip_df['client_id'].nunique()
b = trip_df['driver_id'].nunique()
print(f'number of user is {a}')
print(f'number of driver is {b}')

number of user is 700
number of driver is 300


In [42]:
trip_df.head()

,trip_id,pickup_datetime,dropoff_datetime,PU_loc_id,DO_loc_id,payment_id,driver_id,client_id,rating
0,1,2024-01-01 00:57:55,2024-01-01 01:17:43,186,79,1,27,88,negative
1,2,2024-01-01 00:03:00,2024-01-01 00:09:36,140,236,2,278,499,neutral
2,3,2024-01-01 00:17:06,2024-01-01 00:35:01,236,79,3,202,486,negative
3,4,2024-01-01 00:36:38,2024-01-01 00:44:56,79,211,4,30,65,neutral
4,5,2024-01-01 00:46:51,2024-01-01 00:52:57,211,148,5,258,294,negative


In [43]:
trip_df_new = trip_df[['trip_id', 'client_id', 'driver_id','payment_id', 'PU_loc_id', 'DO_loc_id', 'rating', 'pickup_datetime', 'dropoff_datetime']].copy()

In [44]:
trip_df_new.head()

,trip_id,client_id,driver_id,payment_id,PU_loc_id,DO_loc_id,rating,pickup_datetime,dropoff_datetime
0,1,88,27,1,186,79,negative,2024-01-01 00:57:55,2024-01-01 01:17:43
1,2,499,278,2,140,236,neutral,2024-01-01 00:03:00,2024-01-01 00:09:36
2,3,486,202,3,236,79,negative,2024-01-01 00:17:06,2024-01-01 00:35:01
3,4,65,30,4,79,211,neutral,2024-01-01 00:36:38,2024-01-01 00:44:56
4,5,294,258,5,211,148,negative,2024-01-01 00:46:51,2024-01-01 00:52:57


In [45]:
df_zone = df_zone.drop_duplicates()

In [46]:
df_zone.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [47]:
# Create zone dataset
zone_df = df_zone[['LocationID', 'Borough', 'Zone']].copy()

In [48]:
zone_df = zone_df.rename({'LocationID': 'location_id'}, axis=1)

In [49]:
zone_df.head()

,location_id,Borough,Zone
0,1,EWR,Newark Airport
1,2,Queens,Jamaica Bay
2,3,Bronx,Allerton/Pelham Gardens
3,4,Manhattan,Alphabet City
4,5,Staten Island,Arden Heights


In [50]:
# Save dataset into csv format
vehicle_df.to_csv('out_vehicles.csv', index=False)
user_df.to_csv('out_users.csv', index=False)
driver_df.to_csv('out_drivers.csv', index=False)
zone_df.to_csv('out_location.csv', index=False)
trip_df_new.to_csv('out_trips.csv', index=False)
payment_df.to_csv('out_payment.csv', index=False)